In [2]:
!pip install -U finance-datareader

You should consider upgrading via the 'c:\python\python36\python.exe -m pip install --upgrade pip' command.


In [3]:
import sqlite3
import pandas as pd
import FinanceDataReader as fdr

In [4]:
df_krx = fdr.StockListing("KRX")

In [5]:
def item_code_by_item_name(item_name):
    item_code_list = df_krx.loc[df_krx["Name"] == item_name, "Symbol"].tolist()
    if len(item_code_list) > 0:
        item_code = item_code_list[0]
        return item_code
    else:
        return False

In [6]:
conn = sqlite3.connect('./pred_db.sqlite3')
c = conn.cursor()
datas = pd.read_sql('SELECT * FROM article', con=conn)
c.close()

In [7]:
datas

,id,date,time,title,content,press,posi_nega,stock,label,pred
0,1,2021-08-11,15:39,북한 한미연합훈련 맹비난에 경협주 하락(종합),[그래픽] 최근 남북관계 주요 일지(서울=연합뉴스) 이재윤 기자 = 김영철 노동당 ...,연합뉴스,posi_nega,stock,0,부정
1,2,2021-08-11,15:36,"원티드랩, 코스닥 상장 첫날 '따상' 성공(종합)",원티드랩[프레인키웨스트 제공. 재판매 및 DB금지](서울=연합뉴스) 이미령 기자 =...,연합뉴스,posi_nega,원티드랩,2,긍정
2,3,2021-08-11,14:38,가비아 COM 도메인 다년 연장 이벤트,9월 1일부터 .com 도메인 7% 인상향후 8년에 걸쳐 인상 예정[가비아 제공][...,헤럴드경제,posi_nega,가비아,1,중립
3,4,2021-08-11,14:37,"한솔로지스틱스, 해상운임 급등에 순익 8배…재평가 기대",한솔로지스틱스가 강세다. 2분기 실적 개선 소식이 주가에 영향을 준 것으로 보인다....,아시아경제,posi_nega,한솔로지스틱스,2,긍정
4,5,2021-08-11,14:19,"엔지켐생명과학, 경구형 코로나 치료제 호주서 상용화 첫발 소식에 강세",[파이낸셜뉴스]코로나19 4차 대유행 속 게임체인저로 꼽히는 머크의 코로나19 치료...,파이낸셜뉴스,posi_nega,엔지켐생명과학,1,긍정
...,...,...,...,...,...,...,...,...,...,...
16164,7990,2019-02-27,09:14,막 오르는 2차 북미 정상회담 …경협주도 연일 들썩,[머니투데이 진경진 기자] 2차 북·미 정상회담을 맞아 남북 경협 관련 주식들이 연...,머니투데이,posi_nega,stock,1,중립
16165,7991,2019-02-27,09:14,"바이로메드, 임상결과 지연 소식에 하락세",[이데일리 이광수 기자] 바이로메드(084990)가 주력 파이프라인인 VM202-D...,이데일리,posi_nega,stock,0,부정
16166,7992,2019-02-27,09:13,"에이치엔티, 300억원 투자유치 소식에 상한가",[머니투데이 김사무엘 기자] 에이치엔티가 총 300억원 규모의 전환사채 발생과 유상...,머니투데이,posi_nega,에이치엔티,2,긍정
16167,7993,2019-02-27,09:13,"에이치엔티, 300억 CB·유증 발행 소식에 상한가",[이데일리 이명철 기자] 최대주주 변경 계약을 맺은 에이치엔티(176440)가 대규...,이데일리,posi_nega,에이치엔티,2,긍정


In [8]:
weight = list()
for i in range(0, len(datas)):
    try:
        stock = item_code_by_item_name(datas.loc[i]['stock'])
        date = datas.loc[i]['date'].replace('-','')
        change = fdr.DataReader(stock, date, date)['Change']
        
        pred = datas.loc[i]['pred']
        
        if change[0] == 'nan':
            weight.append('nan')
        elif change[0] > 0 and pred == '긍정':
            weight.append(1)
        elif change[0] < 0 and pred == '부정':
            weight.append(1)
        else:
            weight.append(0)
    except:
        weight.append('nan')
        
weight

['nan',
 0,
 0,
 1,
 1,
 1,
 1,
 'nan',
 1,
 'nan',
 0,
 0,
 1,
 'nan',
 1,
 1,
 1,
 'nan',
 1,
 1,
 1,
 1,
 1,
 'nan',
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 'nan',
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 'nan',
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 'nan',
 1,
 1,
 1,
 0,
 1,
 'nan',
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 'nan',
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 'nan',
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 'nan',
 0,
 'nan',
 'nan',
 'nan',
 1,
 1,
 'nan',
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 'nan',
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 'nan',
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 'nan',
 'nan',
 0,
 1,
 0,
 1,
 1,
 0,
 'nan',
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 'nan',
 1,
 0,
 1,


In [9]:
datas_copy = datas[['id','press', 'pred','date']].copy()
datas_copy['wei'] = weight
datas_copy

,id,press,pred,date,wei
0,1,연합뉴스,부정,2021-08-11,nan
1,2,연합뉴스,긍정,2021-08-11,0
2,3,헤럴드경제,중립,2021-08-11,0
3,4,아시아경제,긍정,2021-08-11,1
4,5,파이낸셜뉴스,긍정,2021-08-11,1
...,...,...,...,...,...
16164,7990,머니투데이,중립,2019-02-27,nan
16165,7991,이데일리,부정,2019-02-27,nan
16166,7992,머니투데이,긍정,2019-02-27,1
16167,7993,이데일리,긍정,2019-02-27,1


In [10]:
all_dict = dict()
press_weight = dict()

for i in range(0, len(datas_copy)):
    
    key = datas_copy.loc[i]['press']
    wei = datas_copy.loc[i]['wei']
    date = datas_copy.loc[i]['date']
    
    if date not in all_dict:
        press_weight = dict()
        if wei == 1:
            if key not in press_weight:
                press_weight[key] = [0, 0]
            else:
                press_weight[key][0] += 1
                press_weight[key][1] += 1
        elif wei == 0:
            if key not in press_weight:
                press_weight[key] = [0, 0]
            else:
                press_weight[key][1] += 1
        all_dict[date] = press_weight
        
    else:
        if wei == 1:
            if key not in press_weight:
                press_weight[key] = [0, 0]
            else:
                press_weight[key][0] += 1
                press_weight[key][1] += 1
        elif wei == 0:
            if key not in press_weight:
                press_weight[key] = [0, 0]
            else:
                press_weight[key][1] += 1
        all_dict[date] = press_weight
        
all_dict

{'2021-08-11': {'연합뉴스': [0, 2],
  '헤럴드경제': [0, 0],
  '아시아경제': [6, 7],
  '파이낸셜뉴스': [2, 4],
  '이데일리': [8, 11]},
 '2021-08-10': {'연합뉴스': [1, 1],
  '파이낸셜뉴스': [3, 4],
  '이데일리': [5, 9],
  '아시아경제': [4, 6]},
 '2021-08-09': {'연합뉴스': [3, 3],
  '파이낸셜뉴스': [4, 6],
  '아시아경제': [6, 10],
  '이데일리': [4, 6]},
 '2021-08-06': {'연합뉴스': [0, 0],
  '파이낸셜뉴스': [0, 1],
  '아시아경제': [2, 4],
  '이데일리': [2, 3]},
 '2021-08-05': {'연합뉴스': [2, 2],
  '파이낸셜뉴스': [4, 5],
  '아시아경제': [4, 5],
  '이데일리': [5, 7]},
 '2021-08-04': {'연합뉴스': [1, 2],
  '파이낸셜뉴스': [9, 11],
  '아시아경제': [5, 6],
  '이데일리': [7, 8]},
 '2021-08-03': {'아시아경제': [6, 8],
  '파이낸셜뉴스': [1, 3],
  '이데일리': [6, 9],
  '연합뉴스': [0, 0]},
 '2021-08-02': {'연합뉴스': [0, 0],
  '파이낸셜뉴스': [1, 1],
  '이데일리': [4, 6],
  '아시아경제': [5, 7]},
 '2021-07-30': {'연합뉴스': [2, 2],
  '헤럴드경제': [0, 0],
  '아시아경제': [6, 10],
  '이데일리': [2, 4],
  '파이낸셜뉴스': [1, 1]},
 '2021-07-29': {'연합뉴스': [0, 2],
  '파이낸셜뉴스': [2, 4],
  '아시아경제': [3, 4],
  '이데일리': [4, 4]},
 '2021-07-28': {'연합뉴스': [0, 2],
  '아시아경제': [3, 6],
  '파이낸셜

In [11]:
all_data = dict()

for key, value in all_dict.items():
    small_data = dict()
    for key2, value2 in value.items():
        try:
            small_data[key2] = int(value[key2][0]/value[key2][1]*100)
            all_data[key] = small_data
        except:
            small_data[key2] = 0
all_data

{'2021-08-11': {'연합뉴스': 0, '헤럴드경제': 0, '아시아경제': 85, '파이낸셜뉴스': 50, '이데일리': 72},
 '2021-08-10': {'연합뉴스': 100, '파이낸셜뉴스': 75, '이데일리': 55, '아시아경제': 66},
 '2021-08-09': {'연합뉴스': 100, '파이낸셜뉴스': 66, '아시아경제': 60, '이데일리': 66},
 '2021-08-06': {'연합뉴스': 0, '파이낸셜뉴스': 0, '아시아경제': 50, '이데일리': 66},
 '2021-08-05': {'연합뉴스': 100, '파이낸셜뉴스': 80, '아시아경제': 80, '이데일리': 71},
 '2021-08-04': {'연합뉴스': 50, '파이낸셜뉴스': 81, '아시아경제': 83, '이데일리': 87},
 '2021-08-03': {'아시아경제': 75, '파이낸셜뉴스': 33, '이데일리': 66, '연합뉴스': 0},
 '2021-08-02': {'연합뉴스': 0, '파이낸셜뉴스': 100, '이데일리': 66, '아시아경제': 71},
 '2021-07-30': {'연합뉴스': 100,
  '헤럴드경제': 0,
  '아시아경제': 60,
  '이데일리': 50,
  '파이낸셜뉴스': 100},
 '2021-07-29': {'연합뉴스': 0, '파이낸셜뉴스': 50, '아시아경제': 75, '이데일리': 100},
 '2021-07-28': {'연합뉴스': 0, '아시아경제': 50, '파이낸셜뉴스': 50, '이데일리': 50},
 '2021-07-27': {'연합뉴스': 0, '파이낸셜뉴스': 16, '아시아경제': 75, '이데일리': 0},
 '2021-07-26': {'아시아경제': 71, '파이낸셜뉴스': 100, '이데일리': 100},
 '2021-07-23': {'연합뉴스': 0, '아시아경제': 40, '이데일리': 100},
 '2021-07-22': {'연합뉴스': 33, '헤럴드경제': 0, '파

In [12]:
per_d = list()
date_k = all_data.keys()

for date in date_k:
    for press in (all_data[date]):
        per_d.append([date, press, all_data[date][press]])
        
per = pd.DataFrame(per_d, columns=['date','press','percentage'])
per

,date,press,percentage
0,2021-08-11,연합뉴스,0
1,2021-08-11,헤럴드경제,0
2,2021-08-11,아시아경제,85
3,2021-08-11,파이낸셜뉴스,50
4,2021-08-11,이데일리,72
...,...,...,...
3231,2019-02-27,이데일리,100
3232,2019-02-27,아시아경제,66
3233,2019-02-27,조선비즈,100
3234,2019-02-27,연합뉴스,100


In [13]:
con = sqlite3.connect('./percentage.sqlite3')
c = con.cursor()
per.to_sql('percentage', con, index=False)
c.close()

In [14]:
conn = sqlite3.connect('./percentage.sqlite3')
c = conn.cursor()
per_data = pd.read_sql('SELECT * FROM percentage', con=conn)
c.close()

In [15]:
per_data

,date,press,percentage
0,2021-08-11,연합뉴스,0
1,2021-08-11,헤럴드경제,0
2,2021-08-11,아시아경제,85
3,2021-08-11,파이낸셜뉴스,50
4,2021-08-11,이데일리,72
...,...,...,...
3231,2019-02-27,이데일리,100
3232,2019-02-27,아시아경제,66
3233,2019-02-27,조선비즈,100
3234,2019-02-27,연합뉴스,100
